In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing_6_sites.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df = df.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
df = df.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
df = df.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
df = df.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
df = df.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.5,SO2.5,NO2.5,CO.5,O3.5,TEMP.5,PRES.5,DEWP.5,RAIN.5,WSPM.5
23,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,28.0,14.0,20.0,1100.0,44.0,0.2,1028.6,-17.6,0.0,1.1
24,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
25,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
29,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
31,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6


In [5]:
data.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.5,SO2.5,NO2.5,CO.5,O3.5,TEMP.5,PRES.5,DEWP.5,RAIN.5,WSPM.5
count,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,2.324800e+04,23248.000000,23248.000000,...,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000
mean,81.522694,109.706689,16.140662,59.594873,1285.489074,67.536062,55.341872,1.056037e+03,2.410530,0.063244,...,120.422935,14.663403,55.330626,1321.097643,77.251952,59.420518,1009.364215,1.981667,0.067004,1.384334
std,80.547645,94.143278,21.112259,37.482837,1260.612753,905.878139,1170.644511,6.631430e+03,13.609074,0.796334,...,96.820668,19.567869,36.240331,1205.038723,1294.210722,1109.252136,9.945120,13.710552,0.858220,1.175391
min,3.000000,3.000000,1.000000,2.000000,100.000000,0.214200,-9575.000000,9.877000e+02,-35.300000,0.000000,...,2.000000,1.000000,2.000000,100.000000,1.000000,-9525.000000,984.000000,-34.600000,0.000000,0.000000
25%,21.000000,37.000000,3.000000,30.000000,500.000000,8.000000,2.900000,1.004100e+03,-8.700000,0.000000,...,45.000000,2.000000,26.000000,500.000000,10.000000,3.100000,1001.200000,-9.400000,0.000000,0.600000
50%,57.000000,86.000000,8.000000,53.000000,900.000000,43.000000,13.800000,1.011800e+03,2.600000,0.000000,...,100.000000,7.000000,49.000000,900.000000,44.000000,14.000000,1008.800000,2.000000,0.000000,1.100000
75%,114.000000,156.000000,20.000000,83.000000,1600.000000,82.000000,22.900000,1.021000e+03,14.900000,0.000000,...,170.000000,19.000000,78.000000,1600.000000,83.000000,23.100000,1017.500000,14.700000,0.000000,1.800000
max,713.000000,948.000000,229.000000,290.000000,10000.000000,78183.000000,28425.000000,1.012125e+06,28.500000,46.400000,...,994.000000,500.000000,271.000000,10000.000000,99603.000000,27725.000000,1038.100000,27.400000,41.900000,12.000000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10.5(t),SO2.5(t),NO2.5(t),CO.5(t),O3.5(t),TEMP.5(t),PRES.5(t),DEWP.5(t),RAIN.5(t),WSPM.5(t)
24,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
25,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
29,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
31,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6
32,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,9.0,22.0,15.0,1200.0,51.0,-0.6,1027.7,-17.8,0.0,0.9


In [7]:
df_forecats_columns = data.loc[:,'PM2.5':'WSPM.5'].columns

In [14]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)']
            df_test = ttest.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)']
            df_original = ttest.loc[:,'PM2.5(t)':'WSPM.5(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            df_original = pd.DataFrame(df_original.values,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [15]:
num_components_pca = 8 #10
order_fts_model = 1
npart = 65
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [16]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 774
--------------------
training window 1548
--------------------
training window 2322
--------------------
training window 3096
--------------------
training window 3870
--------------------
training window 4644
--------------------
training window 5418
--------------------
training window 6192
--------------------
training window 6966


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 7740


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 8514
--------------------
training window 9288
--------------------
training window 10062
--------------------
training window 10836
--------------------
training window 11610
--------------------
training window 12384
--------------------
training window 13158
--------------------
training window 13932
--------------------
training window 14706
--------------------
training window 15480
--------------------
training window 16254
--------------------
training window 17028
--------------------
training window 17802
--------------------
training window 18576
--------------------
training window 19350
--------------------
training window 20124
--------------------
training window 20898
--------------------
training window 21672


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 22446


In [20]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [21]:
print("Statistics MIMO KPCA-NSFTS(test): ")
pd.set_option('display.max_rows', None)
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,21.814,12.568,29.898,0.879,12.088,0.080
1,PM10,33.821,20.041,31.265,0.797,12.619,0.094
2,SO2,5.403,3.130,24.939,0.761,11.103,0.093
3,NO2,13.492,9.053,20.735,0.787,9.084,0.100
4,CO,331.410,203.721,21.371,0.778,9.204,0.094
5,O3,484.421,108.001,5551.162,0.724,21.719,0.119
6,TEMP,296.536,120.886,inf,0.602,18.471,0.114
7,PRES,1.313,0.951,0.094,0.895,0.047,0.063
8,DEWP,1.489,0.931,inf,0.896,10.220,0.076
9,RAIN,0.513,0.083,NaN,-0.105,NaN,NaN


In [22]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_6_sites.csv', index = False, header=True)

In [32]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_6_sites_2.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [23]:
num_components_pca = 20
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [24]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 774
--------------------
training window 1548
--------------------
training window 2322


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 3096
--------------------
training window 3870
--------------------
training window 4644
--------------------
training window 5418
--------------------
training window 6192
--------------------
training window 6966


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 7740


...

--------------------
training window 20898


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 21672


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 22446


In [25]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse
0,PM2.5,26.409,18.392,55.124,0.831,19.475,0.097
1,PM10,40.029,27.799,54.210,0.741,19.282,0.108
2,SO2,7.009,4.722,52.754,0.584,21.878,0.124
3,NO2,18.073,13.235,33.014,0.636,14.210,0.131
4,CO,399.747,289.796,35.058,0.705,15.405,0.111
5,O3,442.399,184.669,3986.647,0.649,31.736,0.142
6,TEMP,320.616,206.269,inf,0.679,19.083,0.121
7,PRES,1.893,1.353,0.134,0.801,0.067,0.098
8,DEWP,2.224,1.587,inf,0.753,14.687,0.113
9,RAIN,0.535,0.219,inf,-0.272,97.832,NaN


In [26]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_6_sites.csv', index = False, header=True)

In [42]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_6_sites_2.csv', index = False, header=True)

In [15]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2
0,PM2.5(t),24.737,17.252,58.239,0.788
1,PM10(t),35.035,24.788,49.847,0.666
2,SO2(t),6.852,4.883,54.403,0.656
3,NO2(t),16.264,11.991,34.577,0.635
4,CO(t),441.296,312.880,40.486,0.667
5,O3(t),329.866,145.893,1567.584,0.628
6,TEMP(t),237.855,163.357,inf,0.632
7,PRES(t),4328.199,1774.043,151.272,0.709
8,DEWP(t),1.982,1.420,inf,0.812
9,RAIN(t),0.302,0.145,inf,-0.173
